# Bearbejdning/"pre-processing" af tekst (tokenization)

At arbejde med hele rå strings er for det meste ikke særlig praktisk.

- Søgning på substrings matcher også, hvis det indgår inde i et ord (medmindre man bruger regular expression, som vi ikke kommer ind på her)
- Vanskeligt at identificere nøgleord / de mest brugte eller væsentligste ord i teksten
- Python adskiller mellem små og store bogstaver
- Vanskeligt at identificere ord i kontekst

Måden man løser mange disse udfordringer, er ved at behandle ordene i teksterne enkeltvis. På den måde gøres hvert enkelt ord til en analyseenhed. Processen i at omdanne tekst til enkeltord kaldes "tokenization", da de færdigbehandlede ord referes til som tokens. 

## Tokenization af tekst

Formålet med tokenization er at konvertere tekst til enkelte analyseenheder. I denne proces frasorteres også elementer i teksten, som umiddelbart ikke bidrager med noget meningsfuldt i en analyse af indholdet (fx tegnsætning, hvorvidt ordet er stavet med stort, fyldord som fx stedord osv.)

Tokenization indeholder typisk følgende skridt:

1. Opdeling af tekst i enkeltord
2. Frasortering af tegnsætning
3. Evt. konverter til små bogstaver
4. Frasorter stopord
5. Evt. konverter ord til deres stamme eller navneform

### Tokenization (med basisfunktioner)

Lad os først se på, hvordan vi kan implementere disse skridt enkeltvis med basisfunktioner for at se, hvordan teksten ændres skridt for skridt.

In [1]:
import pandas as pd

tweetdata_url = "https://raw.githubusercontent.com/CALDISS-AAU/course_ndms-I/master/datasets/poltweets_sample.csv"
tweets_df = pd.read_csv(tweetdata_url)

#### 1. Opdeling af tekst i enkeltord

Første skridt i tokenization er at omdanne teksten til en samling af enkeltord. På den måde er hver enkelt ord afgrænset, og vi kan holde styr på deres placering i teksten gennem deres indeks.

In [2]:
tweet = tweets_df.loc[584, 'full_text']
words = tweet.split(" ")

print(tweet)
print("\n")
print(words)

Hjemmeværnet, Politikadetterne og Forsvaret klarer ærterne ved grænserne, så Politiet kan kaste sig over at gøre danskere mere trygge. Sikker på,at @ClausOxfeldt bliver lykkelig for vort krav. Han har jo brugt megen energi på at give DFs grænseindsats skylden for mandskabsmangel. https://t.co/PgenOdeSwn


['Hjemmeværnet,', 'Politikadetterne', 'og', 'Forsvaret', 'klarer', 'ærterne', 'ved', 'grænserne,', 'så', 'Politiet', 'kan', 'kaste', 'sig', 'over', 'at', 'gøre', 'danskere', 'mere', 'trygge.', 'Sikker', 'på,at', '@ClausOxfeldt', 'bliver', 'lykkelig', 'for', 'vort', 'krav.', 'Han', 'har', 'jo', 'brugt', 'megen', 'energi', 'på', 'at', 'give', 'DFs', 'grænseindsats', 'skylden', 'for', 'mandskabsmangel.', 'https://t.co/PgenOdeSwn']


#### 2. Frasortering af tegnsætning

Tegnsætning er typisk "støj" i text mining teknikker, så det frasorteres typisk.

Her frasorteres tegnsætning ved at gå igennem hvert ord med et for loop og derefter gå igennem forskellige tegnsætninger (`punct_list`) i et for loop og erstatte dem med ingenting.

In [3]:
punct_list = [',', '-','.','?','!']

words_nopunct = []

for word in words:
    for punct in punct_list:
        word = word.replace(punct, "")
    words_nopunct.append(word)
    
words_nopunct = list(filter(None, words_nopunct))

print(words_nopunct)

['Hjemmeværnet', 'Politikadetterne', 'og', 'Forsvaret', 'klarer', 'ærterne', 'ved', 'grænserne', 'så', 'Politiet', 'kan', 'kaste', 'sig', 'over', 'at', 'gøre', 'danskere', 'mere', 'trygge', 'Sikker', 'påat', '@ClausOxfeldt', 'bliver', 'lykkelig', 'for', 'vort', 'krav', 'Han', 'har', 'jo', 'brugt', 'megen', 'energi', 'på', 'at', 'give', 'DFs', 'grænseindsats', 'skylden', 'for', 'mandskabsmangel', 'https://tco/PgenOdeSwn']


#### 3. Evt. konverter til små bogstaver

Et primært formål med tokenization er, at ord med samme semantiske betydning bliver identiske. Derfor konverteres ord typisk til små bogstaver for at ensrette tekstmaterialet.

I nogen tilfælde kan det dog give mening at gøre dette med det samme, så man nemmere kan identifcere egenavne.

In [4]:
words_lower = [word.lower() for word in words_nopunct]

print(words_lower)

['hjemmeværnet', 'politikadetterne', 'og', 'forsvaret', 'klarer', 'ærterne', 'ved', 'grænserne', 'så', 'politiet', 'kan', 'kaste', 'sig', 'over', 'at', 'gøre', 'danskere', 'mere', 'trygge', 'sikker', 'påat', '@clausoxfeldt', 'bliver', 'lykkelig', 'for', 'vort', 'krav', 'han', 'har', 'jo', 'brugt', 'megen', 'energi', 'på', 'at', 'give', 'dfs', 'grænseindsats', 'skylden', 'for', 'mandskabsmangel', 'https://tco/pgenodeswn']


#### 4. Frasorter stopord

Et andet formål med tokenization er, at vi står tilbage med analyseenheder (ord), som kan hjælpe os til at sige noget om indholdet i teksten. Alle sprog har ord, som ikke bærer en speciel semantisk betydning. Det kunne fx være stedord (den, det, en, et, jeg, der osv.) og bindeord (og, eller, også, efter, før osv.).

Disse ord kaldes 'stopord'.

Herunder definerer vi vores egen liste af stopord og frasorterer dem med et for loop og en if-betingelse:

In [5]:
stopwords = ['og', 'klarer', 'ved', 'så', 'kan', 'kaste', 'sig', 'over', 'at', 'gøre', 'mere', 'bliver', 
             'for', 'vort', 'han', 'har', 'jo', 'brugt', 'megen', 'på', 'give', 'for']

words_nostop = []

for word in words_lower:
    if word not in stopwords:
        words_nostop.append(word)
        
print(words_nostop)

['hjemmeværnet', 'politikadetterne', 'forsvaret', 'ærterne', 'grænserne', 'politiet', 'danskere', 'trygge', 'sikker', 'påat', '@clausoxfeldt', 'lykkelig', 'krav', 'energi', 'dfs', 'grænseindsats', 'skylden', 'mandskabsmangel', 'https://tco/pgenodeswn']


Udover stopord har vi også noget "twitter-støj"; fx links og brug af @ til at tagge andre brugere.

In [6]:
tokens = []

for word in words_nostop:
    if word.startswith("@") or word.startswith("https"):
        continue
    tokens.append(word)
    
print(tokens)

['hjemmeværnet', 'politikadetterne', 'forsvaret', 'ærterne', 'grænserne', 'politiet', 'danskere', 'trygge', 'sikker', 'påat', 'lykkelig', 'krav', 'energi', 'dfs', 'grænseindsats', 'skylden', 'mandskabsmangel']


#### 5. Evt. konverter ord til deres stamme eller navneform

Formålet med tokenization er både at frasortere støj i teksten og ensrette tokens sådan, at der kun er ét token for alle ord med samme semantiske betydning.

Fx er det uhensigtsmæssigt at behandle "Valgkamp" og "valgkamp" som to forskellige tokens, da det er det samme ord bare med stort og lille forbogstav.

Netop fordi hvert ord som udgangspunkt behanldes som unikt, giver det nogle udfordringer ift. grammatik, da hver ordbøjning bliver hver sit token - medmindre man gør noget ved det!

Derfor er en typisk praksis enten at konvertere ordet til stammen (stemming). Her ville ord som "koste", "koster", "kostede" alle konverteres til "kost", da det er ordets stamme.

En anden praksis er at konvertere til navneformen (lemmatization). Her ville ord som "koste", "koster", "kostede" alle konverteres til "koste", da det er navneform for ordet.

Sådanne konverteringer kræver, at man bruger i forvejen trænede sprogmodeller, da stamme og navneform vil variere fra sprog til sprog. Heldigvis er der udviklet flere af sådanne værktøjer.

Dog er sådan nogen værktøjer ikke uden fejl, da det tager lang tid for en computer at lære, hvad forskellen er på "kost" (noget man spiser), "kost" (noget man fejer med) og "kost" (stammen af verbum "koste").

#### Tokenization som funktion

Det kan være en fordel at sammensætte ens pre-processing i en funktion. På den måde bliver den nemmere at genanvende.

In [7]:
def tokenizer_basic(text):
    punct_list = [',', '-','.','?','!']
    stopwords = ['og', 'klarer', 'ved', 'så', 'kan', 'kaste', 'sig', 'over', 'at', 'gøre', 'mere', 'bliver', 
                 'for', 'vort', 'han', 'har', 'jo', 'brugt', 'megen', 'på', 'give', 'for']

    words = text.split(" ")

    tokens = []

    for word in words:
        if word.startswith("@") or word.startswith("https"):
            continue

        for punct in punct_list:
            word = word.replace(punct, "")

        word = word.lower()

        if word not in stopwords:
            tokens.append(word)


    tokens = list(filter(None, tokens))

    return(tokens)

In [8]:
tokens = tokenizer_basic(tweet)

print(tweet)
print("\n")
print(tokens)

Hjemmeværnet, Politikadetterne og Forsvaret klarer ærterne ved grænserne, så Politiet kan kaste sig over at gøre danskere mere trygge. Sikker på,at @ClausOxfeldt bliver lykkelig for vort krav. Han har jo brugt megen energi på at give DFs grænseindsats skylden for mandskabsmangel. https://t.co/PgenOdeSwn


['hjemmeværnet', 'politikadetterne', 'forsvaret', 'ærterne', 'grænserne', 'politiet', 'danskere', 'trygge', 'sikker', 'påat', 'lykkelig', 'krav', 'energi', 'dfs', 'grænseindsats', 'skylden', 'mandskabsmangel']


### Tokens som pandas series

Ovenstående funktion returnerer en liste af tokens. Lister kan nemt konverteres til en pandas series. På den måde kan man bruge pandas metoder til at arbejde med tekst-værdierne.

I nedenstående laves en hurtig ordoptælling. Det giver selvfølgelig ikke meget mening for så kort en tekst, men ville også fungere på en langt længere tekst.

In [9]:
tokens_series = pd.Series(tokens)

tokens_series.value_counts()

hjemmeværnet        1
påat                1
skylden             1
grænseindsats       1
dfs                 1
energi              1
krav                1
lykkelig            1
sikker              1
politikadetterne    1
trygge              1
danskere            1
politiet            1
grænserne           1
ærterne             1
forsvaret           1
mandskabsmangel     1
dtype: int64